Confirmed by Omar

used car  Price prediction
the dataset: https://www.kaggle.com/datasets/jpayne/852k-used-car-listings/code
Musab


In [ ]:
!pip install kagglehub


In [ ]:
!nvidia-smi

Tue Apr 15 14:43:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import kagglehub
path = kagglehub.dataset_download('jpayne/852k-used-car-listings')

print('Data source import complete.',path)


Data source import complete. /kaggle/input/852k-used-car-listings


In [ ]:
# prompt: import OS

import os
os.listdir('/kaggle/input/852k-used-car-listings')


['tc20171021.csv', 'true_car_listings.csv']

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import accuracy_score, confusion_matrix
import cudf
import cuml
from cuml.linear_model import LogisticRegression
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler


In [ ]:
for dirname, _, filenames in os.walk('tc20171021.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Load the dataset into a cuDF DataFrame
gdf = cudf.read_csv("/kaggle/input/852k-used-car-listings/tc20171021.csv")

# Display the first few rows of the dataset
print(gdf.head())


   Id  Price  Year  Mileage            City State                Vin   Make  \
0   1  16472  2015    18681  Jefferson City    MO  KL4CJBSBXFB267643  Buick   
1   2  15749  2015    27592        Highland    IN  KL4CJASB5FB245057  Buick   
2   3  16998  2015    13650           Boone    NC  KL4CJCSB0FB264921  Buick   
3   4  15777  2015    25195     New Orleans    LA  KL4CJASB4FB217542  Buick   
4   5  16784  2015    22800       Las Vegas    NV  KL4CJBSB3FB166881  Buick   

               Model  
0  EncoreConvenience  
1          EncoreFWD  
2      EncoreLeather  
3          EncoreFWD  
4  EncoreConvenience  


In [ ]:
# exploration
print("\nDataset shape:", gdf.shape)
print("\nSample of the data:")
print(gdf.head())
print("\nClass distribution:")



Dataset shape: (1233042, 9)

Sample of the data:
   Id  Price  Year  Mileage            City State                Vin   Make  \
0   1  16472  2015    18681  Jefferson City    MO  KL4CJBSBXFB267643  Buick   
1   2  15749  2015    27592        Highland    IN  KL4CJASB5FB245057  Buick   
2   3  16998  2015    13650           Boone    NC  KL4CJCSB0FB264921  Buick   
3   4  15777  2015    25195     New Orleans    LA  KL4CJASB4FB217542  Buick   
4   5  16784  2015    22800       Las Vegas    NV  KL4CJBSB3FB166881  Buick   

               Model  
0  EncoreConvenience  
1          EncoreFWD  
2      EncoreLeather  
3          EncoreFWD  
4  EncoreConvenience  

Class distribution:


In [ ]:
gdf = gdf.drop('Vin', axis=1)
gdf = gdf.drop('Id', axis=1)

In [ ]:
print(gdf.head())

   Price  Year  Mileage            City State   Make              Model
0  16472  2015    18681  Jefferson City    MO  Buick  EncoreConvenience
1  15749  2015    27592        Highland    IN  Buick          EncoreFWD
2  16998  2015    13650           Boone    NC  Buick      EncoreLeather
3  15777  2015    25195     New Orleans    LA  Buick          EncoreFWD
4  16784  2015    22800       Las Vegas    NV  Buick  EncoreConvenience


In [ ]:
print("\nNull values per column:")
print(gdf.isnull().sum())



Null values per column:
Price      0
Year       0
Mileage    0
City       0
State      0
Make       0
Model      0
dtype: int64


In [ ]:
# GPU-based label encoding for cuDF
for col in ['State', 'Make', 'Model', 'City']:
    gdf[col] = gdf[col].astype('category').cat.codes

print(gdf.head())


   Price  Year  Mileage  City  State  Make  Model
0  16472  2015    18681  1085     28     7    893
1  15749  2015    27592  1005     19     7    895
2  16998  2015    13650   217     32     7    896
3  15777  2015    25195  1588     22     7    895
4  16784  2015    22800  1201     38     7    893


In [ ]:
pdf_sample = gdf.to_pandas().sample(n=1000000, random_state=511)


In [ ]:
pdf_sample.to_csv('test.csv', index=False)

In [ ]:

import pandas as pd

print("Loading the dataset...")
N_SAMPLES = 100000  # Adjust this based on your available memory
column_names = ['label'] + [f'feature_{i}' for i in range(1, 29)]

# Assuming 'test.csv' is in the current working directory
try:
    # Load the dataset into a pandas DataFrame
    df = pd.read_csv('test.csv', nrows=N_SAMPLES)
    print(df.head())
except FileNotFoundError:
    print("Error: 'test.csv' not found. Please ensure the file exists in the current directory or provide the correct path.")
except pd.errors.ParserError:
    print("Error: Could not parse 'test.csv'. Please check the file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Loading the dataset...
   Price  Year  Mileage  City  State  Make  Model
0   9995  2007    67267  1911     38    55    916
1  21995  2015    46922   257     39    51   1175
2  23993  2016    13589   269     11     9   1709
3  15000  2014    30186  1845     35    55   1470
4  14750  2009   134063    94     49     9   2452


In [ ]:
# Basic data exploration
print("\nDataset shape:", pdf_sample.shape)
print("\nSample of the data:")
print(pdf_sample.head())
print("\nClass distribution:")
print(pdf_sample['Price'].value_counts())


Dataset shape: (1000000, 7)

Sample of the data:
         Price  Year  Mileage  City  State  Make  Model
1219029   9995  2007    67267  1911     38    55    916
1116965  21995  2015    46922   257     39    51   1175
107296   23993  2016    13589   269     11     9   1709
1212314  15000  2014    30186  1845     35    55   1470
499121   14750  2009   134063    94     49     9   2452

Class distribution:
Price
14995    5484
13995    5358
12995    5232
9995     5070
15995    4837
         ... 
97766       1
28573       1
95936       1
53849       1
72450       1
Name: count, Length: 48723, dtype: int64


In [ ]:
X = pdf_sample.drop('Price', axis=1)
y = pdf_sample['Price']

In [ ]:
X

,Year,Mileage,City,State,Make,Model
1219029,2007,67267,1911,38,55,916
1116965,2015,46922,257,39,51,1175
107296,2016,13589,269,11,9,1709
1212314,2014,30186,1845,35,55,1470
499121,2009,134063,94,49,9,2452
...,...,...,...,...,...,...
935184,2016,28557,1800,44,30,837
360866,2014,34178,2370,5,55,2648
305250,2013,101410,1259,6,46,40
964279,2015,21096,379,5,36,500


In [ ]:
y

,Price
1219029,9995
1116965,21995
107296,23993
1212314,15000
499121,14750
...,...
935184,31230
360866,16994
305250,39700
964279,21822


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()

# Fit on training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)


(800000, 6)


In [ ]:
print(X_test_scaled.shape)


(200000, 6)


In [ ]:
print(y_test.shape)


(200000,)


In [ ]:
print(y_train.shape)


(800000,)
